In [1]:
import os
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry\\Kidney-dl\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry\\Kidney-dl'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [5]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_training_config(self)->ModelTrainConfig:
        training=self.config.model_trainer
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,'kidney-ct-scan-image')

        create_directories([Path(training.root_dir)])

        training_config=ModelTrainConfig(
            root_dir= Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path= Path(training.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )

        return training_config


In [10]:
import os
import urllib.request as request
import tensorflow as tf
import time
from zipfile import ZipFile
tf.config.run_functions_eagerly(True)

In [15]:
import tensorflow as tf
from pathlib import Path

class Training:
    def __init__(self, config):
        self.config = config
        # Ensure eager execution is enabled
        tf.config.run_functions_eagerly(True)

    def get_base_model(self):
        # Load the model without compiling it
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path, compile=False)
        # Compile a fresh optimizer for training
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.02),
            loss='categorical_crossentropy',  # replace with your loss if different
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,   # fixed: use float division
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )

        # Validation generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Training generator
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [16]:
try:
    config=ConfigurationManager()
    traing_config=config.get_training_config()
    training=Training(config=traing_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e



[2025-12-10 16:14:48,581- INFO- common - yaml file config\config.yaml loaded sucessfully]
[2025-12-10 16:14:48,584- INFO- common - yaml file params.yaml loaded sucessfully]
[2025-12-10 16:14:48,587- INFO- common - created a directory artifacts]
[2025-12-10 16:14:48,589- INFO- common - created a directory artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


c:\Users\karakkir\OneDrive - Tietoevry\Kidney-dl\kidney-env\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


23/23 ━━━━━━━━━━━━━━━━━━━━ 195s 9s/step - accuracy: 0.7612 - loss: 8.3323 - val_accuracy: 0.7875 - val_loss: 2.8671
[2025-12-10 16:18:03,939- WARNING- saving_api - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
